# Weapon Detection Model Training
This notebook trains a TensorFlow object detection model using your custom dataset

## 1. Environment Setup
Install required dependencies

In [ ]:
# 1. Install Dependencies
!pip install tensorflow==2.12.0
!pip install tensorflow-object-detection-api
!pip install protobuf==3.20.3
!pip install pycocotools
!pip install pandas

# 2. Clone TF Models
!git clone --depth 1 https://github.com/tensorflow/models

# 3. Compile Protobufs
!cd models/research && protoc object_detection/protos/*.proto --python_out=.



!rmdir /s/q models

# Clean up previous installations
!del setup.py
!del -r training
!del -r ssd_mobilenet*
# 4. Install Object Detection API
!cp models/research/object_detection/packages/tf2/setup.py .
!python -m pip install . --no-deps

# 5. Add to Python Path
import sys
sys.path.append('models/research')
sys.path.append('models/research/object_detection')

# 6. Verify Installation
!python models/research/object_detection/builders/model_builder_tf2_test.py

# 7. Restart Kernel After Installation
print("Please restart the kernel now! (Kernel > Restart Kernel)")

You should consider upgrading via the 'C:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
fatal: destination path 'models' already exists and is not an empty directory.
Could Not Find c:\Users\ih20g\weapon-detection-ai\models\setup.py


## 2. Data Preparation
Parse XML annotations and create TFRecords

In [5]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

# Define class labels based on your folders
CLASSES = {
    'gun': 1,
    'knife': 2,
    'hammer': 3,
    'bat': 4,
    'person': 5
}

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find('filename').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    objects = []
    for obj in root.findall('object'):
        label = obj.find('name').text.lower()
        if label == 'base ball bat':
            label = 'bat'
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        objects.append({
            'label': label,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
        })

    return {
        'filename': filename,
        'width': width,
        'height': height,
        'objects': objects
    }

## 3. Create TFRecords
Convert dataset to TensorFlow format

In [6]:
def create_tf_example(example, images_dir):
    img_path = os.path.join(images_dir, example['filename'])
    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()

    width = example['width']
    height = example['height']
    filename = example['filename'].encode('utf8')
    image_format = b'jpg'

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for obj in example['objects']:
        xmins.append(obj['xmin'] / width)
        xmaxs.append(obj['xmax'] / width)
        ymins.append(obj['ymin'] / height)
        ymaxs.append(obj['ymax'] / height)
        classes_text.append(obj['label'].encode('utf8'))
        classes.append(CLASSES[obj['label']])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [8]:
# Write TFRecord files
def generate_tfrecord_from_dataset(dataset_dir, output_path):
    writer = tf.io.TFRecordWriter(output_path)

    for class_folder in os.listdir(dataset_dir):
        folder_path = os.path.join(dataset_dir, class_folder)
        if not os.path.isdir(folder_path):
            continue

        for file in os.listdir(folder_path):
            if not file.endswith(".xml"):
                continue
            xml_path = os.path.join(folder_path, file)
            try:
                annotation = parse_xml(xml_path)
                tf_example = create_tf_example(annotation, folder_path)
                writer.write(tf_example.SerializeToString())
            except Exception as e:
                print(f"Skipping {xml_path}: {e}")

    writer.close()

# Example usage
#generate_tfrecords_from_dataset('images/train', 'annotations/train', 'train.record')
#generate_tfrecords_from_dataset('images/val', 'annotations/val', 'val.record')

## 4. Generate Label Map

In [9]:
!mkdir -p training
label_map_content = ""
for name, id in CLASSES.items():
    label_map_content += f'item {{\n  id: {id}\n  name: \"{name}\"\n}}\n\n'

with open('training/label_map.pbtxt', 'w') as f:
    f.write(label_map_content)

## 5. Model Configuration

In [4]:
import os
import urllib.request
import tarfile
import shutil

# 1. Download the model
model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
model_tar = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

if not os.path.exists(model_tar):
    print("Downloading model...")
    urllib.request.urlretrieve(model_url, model_tar)
else:
    print("Model already downloaded.")

# 2. Extract the tar.gz
model_dir = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
if not os.path.exists(model_dir):
    print("Extracting model...")
    with tarfile.open(model_tar) as tar:
        tar.extractall()
else:
    print("Model already extracted.")

# 3. Move 'checkpoint' folder to 'training/checkpoint'
os.makedirs("training", exist_ok=True)
src_checkpoint = os.path.join(model_dir, "checkpoint")
dst_checkpoint = os.path.join("training", "checkpoint")

if not os.path.exists(dst_checkpoint):
    shutil.move(src_checkpoint, dst_checkpoint)
    print("Checkpoint moved to training/")
else:
    print("Checkpoint already exists.")
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# !tar -xzf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint training/

Model already downloaded.
Model already extracted.
Checkpoint already exists.


In [5]:
# # Modify the pipeline.config
# !sed -i "s/num_classes: 90/num_classes: 5/g" ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
# !sed -i "s|PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt|training/label_map.pbtxt|g" ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
# !sed -i "s|PATH_TO_BE_CONFIGURED/train.record|train.record|g" ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
# !sed -i "s|PATH_TO_BE_CONFIGURED/val.record|val.record|g" ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config


config_path = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"

# Read the config file
with open(config_path, "r") as f:
    config = f.read()

# Perform replacements
config = config.replace("num_classes: 90", "num_classes: 5")
config = config.replace("PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt", "training/label_map.pbtxt")
config = config.replace("PATH_TO_BE_CONFIGURED/train.record", "train.record")
config = config.replace("PATH_TO_BE_CONFIGURED/val.record", "val.record")

# Save the updated config
with open(config_path, "w") as f:
    f.write(config)

print("✅ pipeline.config updated successfully.")

✅ pipeline.config updated successfully.


## 6. Start Training

In [6]:
!python -m object_detection.model_main_tf2 \
    --pipeline_config_path=ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
    --model_dir=training/ \
    --alsologtostderr \
    --num_train_steps=10000 \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=500

c:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe: No module named object_detection.model_main_tf2


## 7. Export Trained Model

In [ ]:
# import os
# import sys
# sys.path.append(os.path.abspath('models/research'))
# sys.path.append(os.path.abspath('models/research/slim'))

# !python -m object_detection.exporter_main_v2 \
#     --input_type image_tensor \
#     --pipeline_config_path ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
#     --trained_checkpoint_dir training/ \
#     --output_directory inference_graph

import os
import sys

# Add model directories to sys.path
sys.path.append(os.path.abspath('models/research'))
sys.path.append(os.path.abspath('models/research/slim'))
sys.path.append(os.path.abspath('models/research/object_detection'))

# Run the exporter module
!python -m object_detection.exporter_main_v2 \
    --input_type image_tensor \
    --pipeline_config_path ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
    --trained_checkpoint_dir training/ \
    --output_directory inference_graph

c:\Users\ih20g\AppData\Local\Programs\Python\Python310\python.exe: No module named exporter_main_v2


## 8. Verify Model

In [ ]:
import cv2
import numpy as np
from object_detection.utils import visualization_utils as viz_utils

# Load saved model
model = tf.saved_model.load('inference_graph/saved_model')

def detect_objects(image_path):
    image = cv2.imread(image_path)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    detections = model(input_tensor)

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(int),
        detections['detection_scores'][0].numpy(),
        label_map_util.create_category_index_from_labelmap('training/label_map.pbtxt'),
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.5
    )

    cv2.imshow('Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test on a sample image
detect_objects('test_images/image_401.jpg')